In [1]:
# Use centrailzed training to compare with federated learning
epochs = 20
n_train_items = 1280

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import numpy as np


In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 3
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [10]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

train_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
# test_dataloader = [
#         ((data), (target))
#         for i, (data, target) in enumerate(test_loader)
#         if i < n_train_items / args.batch_size
#     ]


In [25]:
print(len(test_dataloader))
#len(list(test_dataloader.__iter__())[0][1])

20


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [12]:
def train(args, device, model, train_dataloader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_dataloader) * args.batch_size,
                100. * batch_idx / len(train_dataloader), loss.item()))

In [27]:
def test(args, device, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader)*(args.batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader)* (args.batch_size),
        100. * correct / (len(test_loader)*args.batch_size)))

In [28]:
#%%time
model = Net().to(device)
optimizer =  optim.SGD(model.parameters(), lr=args.lr)
for epoch in range(1, args.epochs + 1):
    train(args, device, model, train_dataloader, optimizer, epoch)
    test(args, device, model, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/1280 (0%)]	Loss: 2.314400
Train Epoch: 1 [192/1280 (15%)]	Loss: 2.324195
Train Epoch: 1 [384/1280 (30%)]	Loss: 2.298956
Train Epoch: 1 [576/1280 (45%)]	Loss: 2.287949
Train Epoch: 1 [768/1280 (60%)]	Loss: 2.273747
Train Epoch: 1 [960/1280 (75%)]	Loss: 2.279161
Train Epoch: 1 [1152/1280 (90%)]	Loss: 2.265763

Test set: Average loss: 2.2627, Accuracy: 2135/10048 (21%)

Train Epoch: 2 [0/1280 (0%)]	Loss: 2.269754
Train Epoch: 2 [192/1280 (15%)]	Loss: 2.276045
Train Epoch: 2 [384/1280 (30%)]	Loss: 2.257073
Train Epoch: 2 [576/1280 (45%)]	Loss: 2.252402
Train Epoch: 2 [768/1280 (60%)]	Loss: 2.235370
Train Epoch: 2 [960/1280 (75%)]	Loss: 2.230212
Train Epoch: 2 [1152/1280 (90%)]	Loss: 2.214319

Test set: Average loss: 2.2169, Accuracy: 4117/10048 (41%)

Train Epoch: 3 [0/1280 (0%)]	Loss: 2.219200
Train Epoch: 3 [192/1280 (15%)]	Loss: 2.219878
Train Epoch: 3 [384/1280 (30%)]	Loss: 2.203995
Train Epoch: 3 [576/1280 (45%)]	Loss: 2.204735
Train Epoch: 3 [768/1280 (60%)]	Loss: 2